# Constructing Features from Light Curves

In this notebook, we calculate the root mean square variability, the peak-to-peak amplitude, the flux standard deviation, the color indices and the peak lag and peak correlation for all light curves in our data file and save them together in a single data frame which contains a row where all the features relating only to the specific object is stored.

## The Features
* **Root-mean-square (RMS) variability in g and r**: we calculate the RMS variability in both g and r bands separately, to get a sense of the AGN’s typical variability amplitude.
* **Amplitude in g and r**: we compute the peak-to-peak amplitude as the difference between the maximum and minimum flux. We also computed the flux standard deviation in each band.
* **Colour index**: the difference between the average g and r band magnitudes.
* **F-variability**: Gives a measure of the intrinsic variability of a light curve after correcting for observational uncertainties. It is computed with the variance of the magnitude, the mean squared error and the average magnitude.
* **Peak lag and peak correlation**: Peak lag is the time delay between the variations in two light curves (in this case, g-band and r-band). Peak correlation quantifies the strength of the relationship between the variations in two light curves (g-band and r-band) at the lag time where the correlation is highest.


In [14]:
#import necessary packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import gc
from scipy.signal import find_peaks
from scipy.signal import correlate

In [15]:
#directory
data_dir = '/home/shoaib/PSChallenge/'

In [16]:
#load up filenames
files=[]
for filename in os.listdir(data_dir + 'Lightcurves_by_Name/'):
  if filename.endswith('.csv'):
    files.append(filename)
print(files)

['[MML2015] 5BZQ J0435+2532.csv', 'SDSS J014835.65+194152.2.csv', 'SDSS J231606.34+285222.4.csv', 'SDSS J124450.26+214702.6.csv', 'SDSS J110237.65+055202.9.csv', 'SDSS J091118.02+202254.7.csv', 'SDSS J235806.85+282852.0.csv', 'SDSS J143243.53+515105.7.csv', 'SDSS J001007.73+081132.8.csv', 'SDSS J080823.67+264608.3.csv', 'SDSS J165454.94+294442.6.csv', 'SDSS J120237.14+665247.4.csv', 'SDSS J165936.51+313905.0.csv', 'SDSS J114111.99+392555.9.csv', 'SDSS J113923.79+644346.0.csv', 'SDSS J130950.07+591423.3.csv', 'SDSS J003921.40-031824.7.csv', 'SDSS J093200.84+145413.1.csv', '[MML2015] 5BZQ J0130-1019.csv', '[MML2015] 5BZQ J1146+3958.csv', 'SBS 1036+490.csv', 'SDSS J162143.15+270959.7.csv', 'SDSS J235328.57+102526.6.csv', 'SDSS J115255.22+610604.7.csv', 'SDSS J165247.08+434727.5.csv', 'SDSS J094815.96-015038.1.csv', 'SDSS J090601.44+463041.5.csv', 'SDSS J155049.35+553744.3.csv', 'SDSS J171117.16+391516.7.csv', 'SDSS J153241.35+384409.8.csv', 'SDSS J141139.00+562105.9.csv', 'SDSS J103534.03

In [17]:
# define our columns
features = ['name', 'rms_g', 'rms_r', 'amplitude_g', 'amplitude_r', 'mag_std_g', 'mag_std_r', 'f_var_g', 'f_var_r', 'color_index', 'peak_lag', 'peak_correlation']
df_features = pd.DataFrame(columns=features)

# Methods to Compute Features

## RMS Variability
We calculate the RMS variability in both g and r bands separately, to get a sense of the AGN’s typical variability amplitude.


In [18]:
def calculate_rms_variability(light_curve):
    """
    Calculate RMS variability for a light curve.
    
    Parameters:
    light_curve (pd.DataFrame): A DataFrame containing 'mjd', 'mag', and 'magerr' columns.
    
    Returns:
    float: The RMS variability of the light curve.
    """
    # Extract magnitudes and magnitude errors
    mag = light_curve['mag']
    magerr = light_curve['magerr']
    
    # Weights are inversely proportional to the square of the magnitude errors
    weights = 1 / magerr**2
    
    # Calculate the weighted mean of the magnitudes
    weighted_mean_mag = np.sum(weights * mag) / np.sum(weights)
    
    # Calculate deviations from the weighted mean
    deviations = mag - weighted_mean_mag
    
    # Compute the RMS variability
    rms_variability = np.sqrt(np.sum(weights * deviations**2) / np.sum(weights))
    
    return rms_variability

## Peak-to-peak amplitude

We compute the peak-to-peak amplitude as the difference between the maximum and minimum flux.

In [19]:
def calculate_peak_to_peak_amplitude(light_curve):
    """
    Calculate the peak-to-peak amplitude of a light curve.
    
    Parameters:
    light_curve (pd.DataFrame): A DataFrame containing 'mjd', 'mag', and 'magerr' columns.
    
    Returns:
    float: The peak-to-peak amplitude of the light curve.
    """
    # Extract magnitudes
    mag = light_curve['mag'].values
    
    # Find peaks (local maxima)
    peaks, _ = find_peaks(mag)
    
    # Find valleys (local minima) by inverting the magnitude
    valleys, _ = find_peaks(-mag)
    
    if len(peaks) == 0 or len(valleys) == 0:
        # If no peaks or valleys are detected, return 0 as amplitude
        return 0
    
    # Get the highest peak and the lowest valley
    max_peak = mag[peaks].max()
    min_valley = mag[valleys].min()
    
    # Calculate the peak-to-peak amplitude
    amplitude = max_peak - min_valley
    
    return amplitude


## Colour Index

Is the difference between the average g and r band magnitudes.

In [20]:
def calculate_color_index(lc_g, lc_r):
    """
    Calculate the color index from g-band and r-band light curves.
    
    Parameters:
    lc_g (pd.DataFrame): A DataFrame containing 'mjd', 'mag', and 'magerr' for the g-band.
    lc_r (pd.DataFrame): A DataFrame containing 'mjd', 'mag', and 'magerr' for the r-band.
    
    Returns:
    float: The color index (mean_g - mean_r).
    """
    # Calculate the mean magnitude in each band
    mean_g = np.mean(lc_g['mag'])
    mean_r = np.mean(lc_r['mag'])
    
    # Compute the color index
    color_index = mean_g - mean_r
    
    return color_index

## F-Variability

Gives a measure of the intrinsic variability of a light curve after correcting for observational uncertainties. It is computed with the variance of the magnitude, the mean squared error and the average magnitude.


In [21]:
def calculate_f_variability(light_curve):
    """
    Calculate the F-variability of a light curve.
    
    Parameters:
    light_curve (pd.DataFrame): A DataFrame containing 'mjd', 'mag', and 'magerr' columns.
    
    Returns:
    float: The F-variability of the light curve. If the result is invalid, returns NaN.
    """
    
    # Extract magnitudes and magnitude errors
    mag = light_curve['mag'].values
    magerr = light_curve['magerr'].values
    
    # Mean magnitude
    mean_mag = np.mean(mag)
    
    if mean_mag == 0:
        # Avoid division by zero
        return np.nan
    
    # Variance of the magnitudes
    variance_mag = np.var(mag, ddof=1)  # Sample variance (unbiased)
    
    # Mean squared error
    mean_squared_error = np.mean(magerr**2)
    
    # Corrected variance (subtract mean squared error from variance)
    corrected_variance = variance_mag - mean_squared_error
    
    if corrected_variance < 0:
        # If corrected variance is negative, F-var is not defined
        return np.nan
    
    # F-variability
    f_variability = np.sqrt(corrected_variance) / mean_mag
    
    return f_variability

## Peak Lag and Correlation

Peak lag is the time delay between the variations in two light curves (in this case, g-band and r-band). Peak correlation quantifies the strength of the relationship between the variations in two light curves (g-band and r-band) at the lag time where the correlation is highest.

In [22]:
def compute_peak_lag_and_correlation(lc_g, lc_r, max_lag=100, num_points=500):
    """
    Compute the peak lag and peak correlation for g-band and r-band light curves.
    
    Parameters:
        lc_g (DataFrame): g-band light curve with 'mjd' and 'mag' columns.
        lc_r (DataFrame): r-band light curve with 'mjd' and 'mag' columns.
        max_lag (int): Maximum lag in days for cross-correlation.
        num_points (int): Number of points for interpolation.
    
    Returns:
        float: Peak lag time (days).
        float: Peak correlation value.
    """
    # Interpolate magnitudes onto a common grid of times
    common_times = np.linspace(
        max(lc_g['mjd'].min(), lc_r['mjd'].min()),
        min(lc_g['mjd'].max(), lc_r['mjd'].max()),
        num=num_points
    )
    mag_g_interp = np.interp(common_times, lc_g['mjd'], lc_g['mag'])
    mag_r_interp = np.interp(common_times, lc_r['mjd'], lc_r['mag'])

    # Subtract the mean to normalize for cross-correlation
    mag_g_interp -= np.mean(mag_g_interp)
    mag_r_interp -= np.mean(mag_r_interp)

    # Compute cross-correlation
    correlation = correlate(mag_g_interp, mag_r_interp, mode='full')
    lags = np.arange(-len(common_times) + 1, len(common_times))

    # Limit lags to the max_lag range
    valid_lags = (lags >= -max_lag) & (lags <= max_lag)
    lags = lags[valid_lags]
    correlation = correlation[valid_lags]

    # Find the lag corresponding to the peak correlation
    peak_idx = np.argmax(correlation)
    peak_lag = lags[peak_idx]
    peak_correlation = correlation[peak_idx]

    return peak_lag, peak_correlation

# Constructing Features from Light Curves

We now run these methods on all light curves in our data folder to compute their features.

In [ ]:
file_num = 0

for file_num in range(0,len(files)):
    print('Computing features for file number: ', file_num, ' of: ', len(files))
    filename=files[file_num]
    this_lc=pd.read_csv('Lightcurves_by_Name/'+filename, index_col=False)
    #retreive name of the LC
    lc_name = filename.rsplit('.csv', 1)[0]
    print('Object name: ', lc_name)

    #COMPUTE THE FEATURES FOR THIS_LC
    # Extract unique 'oid_alerce' values for each band
    oid_g = this_lc.loc[this_lc['band'] == 'g', 'oid_alerce'].unique().tolist()
    oid_r = this_lc.loc[this_lc['band'] == 'r', 'oid_alerce'].unique().tolist()
    
    #create dataframes for g and r (for multiple light curve observations - we ignore the gaps)
    # WE ALSO NEED TO SORT BY MJD
    this_lc_g = this_lc[this_lc['oid_alerce'].isin(oid_g)].reset_index(drop=True)
    this_lc_r = this_lc[this_lc['oid_alerce'].isin(oid_r)].reset_index(drop=True)

    #keep only necessary columns to extract features
    lc_g = this_lc_g[['mjd','mag', 'magerr']].sort_values(by=['mjd'], ascending=True)
    lc_r = this_lc_r[['mjd','mag', 'magerr']].sort_values(by=['mjd'], ascending=True)

    # calculate RMS variability in both bands
    rms_g = calculate_rms_variability(lc_g[['mjd', 'mag', 'magerr']])
    rms_r = calculate_rms_variability(lc_r[['mjd', 'mag', 'magerr']])

    #calculate peak-to-peak amplitude
    amplitude_g = calculate_peak_to_peak_amplitude(lc_g[['mjd', 'mag', 'magerr']])
    amplitude_r = calculate_peak_to_peak_amplitude(lc_r[['mjd', 'mag', 'magerr']])

    # standard deviation
    mag_std_g = lc_g['mag'].std()
    mag_std_r = lc_r['mag'].std()

    #F-variability
    f_var_g = calculate_f_variability(lc_g[['mjd', 'mag', 'magerr']])
    f_var_r = calculate_f_variability(lc_r[['mjd', 'mag', 'magerr']])

    #colour index
    color_index = calculate_color_index(lc_g[['mjd', 'mag', 'magerr']], lc_r[['mjd', 'mag', 'magerr']])

    #peak lag and correlation
    peak_lag, peak_correlation = compute_peak_lag_and_correlation(
        lc_g[['mjd', 'mag', 'magerr']],
        lc_r[['mjd', 'mag', 'magerr']],
        max_lag=10
    )

    # Add all features into a dataframe
    this_features = [lc_name, rms_g, rms_r, amplitude_g, amplitude_r, mag_std_g, mag_std_r, f_var_g, f_var_r, color_index, peak_lag, peak_correlation]
    df_features.loc[len(df_features)] = this_features

#save our constructed features to a complete dataframe
df_features.to_csv(data_dir + 'new_features.csv', index=False)

# Output message
print('All object features have been computed. Saved in new_features.csv')